In [2]:
from conductor.client.ai.ai_orchestrator import AIOrchestrator
from conductor.client.ai.ai_orchestrator import AIConfiguration
from conductor.client.configuration.configuration import Configuration
from conductor.client.configuration.settings.authentication_settings import AuthenticationSettings
from conductor.client.workflow.task.switch_task import SwitchTask

ai_config = AIConfiguration('azure_openai', 'text-davinci-003', 'text-embedding-ada-002', 'pineconedb')
api_config = Configuration(
    authentication_settings=AuthenticationSettings(key_id='3bbcc893-66f4-4887-a419-12b8117fac62', key_secret='t6BhLZUqaKRXLEjzEtnVNnjtO5Ll2C1205SSE1rXg3WrRZ7V'))

prompt_client = AIOrchestrator(api_configuration=api_config, ai_configuration=ai_config)
prompt_client.add_prompt_template('say_hi_to_friend', 'Say hello to your new friend ${friend_name} based on the time of the day.', 'xxx template')
result = prompt_client.test_prompt_template('say_hi_to_friend', {'friend_name': 'viren'})
print(result)



Good morning, Viren! It's nice to meet you.


# Let's try and run a workflow that uses the LLM Chaining

In [4]:
from conductor.client.workflow.conductor_workflow import ConductorWorkflow
from conductor.client.workflow.task.llm_tasks.utils.prompt import Prompt
from conductor.client.workflow.task.llm_tasks.llm_text_complete import LlmTextComplete
from conductor.client.workflow.task.simple_task import SimpleTask
from conductor.client.automator.task_handler import TaskHandler
from conductor.client.worker.worker import Worker
from conductor.client.http.models import Task, TaskResult
from conductor.client.http.models.task_result_status import TaskResultStatus
from conductor.client.worker.worker_interface import WorkerInterface

def get_document(task: Task) -> str:
    return 'Viren Baraiya'

prompt = Prompt(name='say_hi_to_friend', variables={'friend_name': '${get_friend_name_ref.output.result}'})
tasks = [
        SimpleTask('get_friend_name', 'get_friend_name_ref'),
        LlmTextComplete('say_hi', 'say_hi_ref', ai_config.llm_provider, ai_config.text_complete_model, prompt=prompt)
    ]
workflow = ConductorWorkflow(name='say_hi_to_the_friend')
workflow >> tasks
workflow.output_parameters = {'greetings': '${say_hi_ref.output.result}'}

# When running from the notebook
task_executors = {'get_friend_name_ref': get_document}

wf_result = prompt_client.execute_workflow(workflow=workflow, wait_for_seconds=5, task_to_exec=task_executors)
print(wf_result.output)

{'get_friend_name_ref': {'result': 'Viren Baraiya'}, 'say_hi_ref': {'result': "\n\nGood morning, ${friend_name}! It's nice to meet you."}}
